In [27]:
from utils import load_model
import torch

In [3]:
reviser_path = f'pretrained_LCP/Reviser-6-FI/reviser_20/epoch-199.pt'
reviser, _ = load_model(reviser_path, is_local=True, mtl=False)

  [*] Loading model from pretrained_LCP/Reviser-6-FI/reviser_20/epoch-199.pt


In [4]:
dataset_path = f'data/FI_train_tsp/200_FI20_seed1235.pkl'
dataset = reviser.problem.make_dataset(filename=dataset_path, num_samples=200000, offset=0)

In [6]:
dataset.data[0].shape

torch.Size([20, 2])

In [7]:
len(dataset.data)

200000

In [36]:
from utils import reconnect
def load_problem(name):
    from problems import TSP, CVRP, SDVRP, OP, PCTSPDet, PCTSPStoch,LOCAL
    problem = {
        'local': LOCAL,
        'tsp': TSP,
        'cvrp': CVRP,
        'sdvrp': SDVRP,
        'op': OP,
        'pctsp_det': PCTSPDet,
        'pctsp_stoch': PCTSPStoch,
    }.get(name, None)
    assert problem is not None, "Currently unsupported problem: {}!".format(name)
    return problem
problem = load_problem('local')
get_cost_func = lambda input, pi: problem.get_costs(input, pi)


In [58]:
test_sample = dataset[5].unsqueeze(0)
ini_pi = torch.arange(20)
get_cost_func(test_sample, ini_pi)


(tensor([3.2946]), None)

In [60]:
reviser.eval()
reviser.set_decode_type("greedy")
_, pi = reviser(test_sample, return_pi=True)
_, pi

(tensor([3.2336]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 15, 16, 17, 14, 13, 12, 11,
          18, 19]]))

In [61]:
test_sample = test_sample.gather(1, pi.unsqueeze(-1).expand_as(test_sample)).squeeze()


In [69]:
from tqdm import tqdm
dataset_RG = []
bs = 5
for pid in tqdm(range(10)):
    sample = dataset[pid].unsqueeze(0)
    ini_pi = torch.arange(20)
    ini_cost, _ = get_cost_func(sample, ini_pi)

    re_cost, re_pi = reviser(sample, return_pi=True)
    if re_cost < ini_cost:
        re_sample = sample.gather(1, re_pi.unsqueeze(-1).expand_as(sample)).squeeze()
        dataset_RG.append(re_sample)
    else:
        dataset_RG.append(sample.squeeze())



  1%|          | 1858/200000 [00:27<49:11, 67.12it/s]  


KeyboardInterrupt: 